### 사업보고서 "재무제표 주석" 다운로드한 HTML 문서(샘플)에서 값 읽기 테스트

* 여러 파일 테스트

In [5]:
import os

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import mydart

# 샘플 파일 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/sample/"
file_list = os.listdir(path_dir)
 
for file_name in file_list:
    with open(path_dir + file_name, "r") as file:
        html_doc = file.read()
    val_a = val_b = 0.0
    unit = None
    # 재무제표 주석 문서에서 사용된 금액 단위 찾기 (확정급여채무)
    unit = mydart.find_unit(html_doc, url=False)
    # 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
    val_a, val_b = mydart.find_item(html_doc)
    print(file_name, '\t', unit, '\t', val_a, '\t', val_b)   

대원화성_2017.html 	 천원 	 4333527.0 	 0.0
삼성전자_2020.htm 	 백만원 	 9754082.0 	 10916538.0
삼익THK.html 	 원 	 24306779070.0 	 15839581933.0
송원산업_2022.html 	 백만원 	 65649.0 	 -1
이수화학_2017.html 	 천원 	 31658642.0 	 33350365.0
조선내화_2016년.htm 	 천원 	 31474613.0 	 31603167.0
카프로_2016.html 	 천원 	 20364282.0 	 2075400.0
황금에스티_2021.html 	 천원 	 0.0 	 -1


* 한 개 파일 테스트

In [4]:
import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import mydart

# 샘플 파일 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/sample/"
file_name = "이수화학_2017.html"
 
with open(path_dir + file_name, "r") as file:
    html_doc = file.read()
val_a = val_b = 0.0
unit = None
# 재무제표 주석 문서에서 사용된 금액 단위 찾기 (확정급여채무)
unit = mydart.find_unit(html_doc, url=False)
# 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
val_a, val_b = mydart.find_item(html_doc)
print(file_name, '\t', unit, '\t', val_a, '\t', val_b)   

이수화학_2017.html 	 천원 	 31658642.0 	 33350365.0
